In [5]:
%matplotlib inline
%load_ext watermark

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from scipy.io import loadmat
import glob
import os
import pickle
import pandas as pd
from PIL import Image
import time

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [ ]:
## Import EEG and BOLD-fMRI ICA components

good_fmri_ics = np.subtract([10,12,15,23], 0)

sweep_comments = [ [] for _ in range(3) ]

In [ ]:
## Load ICA components figures AUTOMATICALLY

eeg_im_path = '/Users/dmannk/Documents/Lioi2020-XP1/derivatives/figures/'
bold_im_path = '/Users/dmannk/Documents/Lioi2020-XP1/derivatives/group_ica_25ICs_6mm.gica/report/'

eeg_im_names = os.listdir(eeg_im_path)
eeg_im_names.sort()
eeg_ims = []

for im in eeg_im_names:
    im_file = eeg_im_path + im
    print(im_file)
    eeg_ims.append( Image.open(im_file) )

widths = [im.width for im in eeg_ims]
cum_width = np.hstack(( 0 , np.cumsum(widths) ))
tot_width = np.sum(widths)
height = np.mean([im.height for im in eeg_ims]).astype(int)
dst = Image.new('RGB', (tot_width,height) )

for i,im in enumerate(eeg_ims):
    dst.paste(im, (cum_width[i],0) )
    
bold_ics = []

for ic_label in good_fmri_ics:
    im_file = bold_im_path + 'IC_' + str(ic_label+1) + '_thresh.png'
    print(im_file)
    bold_ics.append( Image.open(im_file) )
    
plt.figure(figsize=[9.,3.]); plt.imshow(dst); plt.show(); plt.axis('off')


In [ ]:
freqs = [4,6,8,10,12,16,20,24,28,32,40,50]

In [ ]:
## Import tensHRF sweep results

filename = '/Users/dmannk/Documents/Neurovasc_CC/neurovasc_data_1_16.58_Jan13_2021.pkl'
with open(filename, 'rb') as f:
    #env_eeg_dwn, bold, fmri_fs, ext = pickle.load(f)
    #env_eeg_dwn, bold, fmri_fs, ext, freqs = pickle.load(f)
    env_eeg_dwn, bold, fmri_fs, ext, freqs, good_fmri_ics = pickle.load(f)

filename = '/Users/dmannk/Documents/Neurovasc_CC/Ps_taus_percs_16.58_Jan13_2021.pkl'
with open(filename, 'rb') as f:
    data = pickle.load(f)
    
if len(data) == 10:
    print('Ps, taus and percs NOT YET determined')
    Ps_taus_percs_best, kern, numComps, corrs, corrs_weighted, Ps_taus_percs, HRFs_sweep, Uchans_sweep, Ufreqs_sweep, Usubjs_sweep = data
elif len(data) == 11:
    print('Ps, taus and percs ALREADY determined')
    Ps_taus_percs_best, kern, numComps, corrs, corrs_weighted, Ps_taus_percs, HRFs_sweep, Uchans_sweep, Ufreqs_sweep, Usubjs_sweep, sweep_comments = data
else:
    print('Verify pickle file')
    

In [ ]:
## Plot BOLD group-ICA components

plt.figure(figsize=[24.,36])
for i, im in enumerate(bold_ics): 
    s = plt.subplot(1,bold.shape[-1],i+1); s.set_xticks([]); s.set_yticks([])
    plt.imshow(im)
plt.show(); plt.tight_layout()

In [ ]:
## Plot tensHRF sweep results

for nd in range(len(bold_ics)):

    fig = plt.figure(figsize=[18.,8.])
    s1 = plt.subplot(221)
    s2 = plt.subplot(422)
    s3 = plt.subplot(424)
    s4 = plt.subplot(223)
    s5 = fig.add_subplot(2,9,(15,17))
    s6 = plt.subplot(2,9,18)

    for i in range(len(Ps_taus_percs)):

        plt.suptitle("nd={}   -   i={}   -   {} , {:.2} , {}".format(nd, i, Ps_taus_percs[i][0],Ps_taus_percs[i][1],Ps_taus_percs[i][2]))

        hrfs_df = pd.DataFrame(HRFs_sweep[i,:,:,nd])
        chans_df = pd.DataFrame(Uchans_sweep[i,:,:,nd])
        freqs_df = pd.DataFrame(Ufreqs_sweep[i,:,:,nd], index=freqs)
        subjs_df = pd.DataFrame(Usubjs_sweep[i,:,:,nd])

        hrfs_df.plot(ax=s1, legend=False); plt.xlabel('Time (s)')
        s2.imshow(dst); s2.set_xticks([]); s2.set_yticks([]); s2.axis('off')
        chans_df.plot.bar(rot=0, ax=s3, legend=False); s3.set_xticks([]);
        freqs_df.plot.bar(rot=0, ax=s4, legend=False); plt.xlabel('Frequency (Hz)')
        subjs_df.plot.bar(rot=0, ax=s5, legend=False); plt.xlabel('Subject'); s5.set_title("Weighted correlation: {:}".format(str(corrs_weighted[nd,i,:])))
        subjs_df.plot.hist(ax=s6, orientation="horizontal", legend=False, alpha=0.3); s6.set_yticks([])

        display.clear_output(wait=True)
        display.display(plt.gcf())

        #time.sleep(.5)
        '''
        sweep_comment = input('Write comment for current results, or press Enter to skip ')

        if sweep_comment != '':
            cmt = str(i) + ' ' + str(Ps_taus_percs[i]) + ' ' + str(corrs_weighted[nd,i,:]) + ' - ' + sweep_comment
            sweep_comments[nd].append(cmt)
        '''
        s1.clear(); s2.clear(); s3.clear(); s4.clear(); s5.clear(); s6.clear()

    #plt.draw()
    
    #plt.clf()


In [ ]:
plt.figure()
pd.DataFrame(np.max(corrs, axis=0)).plot.bar(figsize=(17,5), legend=False)
plt.show()

In [ ]:
for nd in range(4):
    tmp = np.where( np.max(corrs, axis=0)[nd,:] == np.max(np.max(corrs, axis=0)[nd,:]) )
    print(tmp)

In [ ]:
## Plot tensHRF sweep results

nd = 3
i = 6

fig = plt.figure(figsize=[18.,8.])
s1 = plt.subplot(221)
s2 = plt.subplot(422)
s3 = plt.subplot(424)
s4 = plt.subplot(223)
s5 = fig.add_subplot(2,9,(15,17))
s6 = plt.subplot(2,9,18)

    
plt.suptitle("i={}   -   {} , {:.2} , {}".format(i, Ps_taus_percs[i][0],Ps_taus_percs[i][1],Ps_taus_percs[i][2]))

hrfs_df = pd.DataFrame(HRFs_sweep[i,:,:,nd])
chans_df = pd.DataFrame(Uchans_sweep[i,:,:,nd])
freqs_df = pd.DataFrame(Ufreqs_sweep[i,:,:,nd], index=freqs)
subjs_df = pd.DataFrame(Usubjs_sweep[i,:,:,nd])

hrfs_df.plot(ax=s1, legend=False); plt.xlabel('Time (s)')
s2.imshow(dst); s2.set_xticks([]); s2.set_yticks([]); s2.axis('off')
chans_df.plot.bar(rot=0, ax=s3, legend=False); s3.set_xticks([]);
freqs_df.plot.bar(rot=0, ax=s4, legend=False); plt.xlabel('Frequency (Hz)')
subjs_df.plot.bar(rot=0, ax=s5, legend=False); plt.xlabel('Subject'); s5.set_title("Weighted correlation: {:}".format(str(corrs_weighted[nd,i,:])))
subjs_df.plot.hist(ax=s6, orientation="horizontal", legend=False); s6.set_yticks([])

plt.draw()


In [ ]:
## Update 'Ps_taus_percs_best'

data[0] = [ () , () , () ]

In [ ]:
## Append 'sweep_comments' to list of saved variables and re-pickle / overwrite file

confirm = input('Type Y to confirm overwrite of Ps_taus_percs pickle file ')

if confirm == 'Y':
    print('Overwriting')
    data.append(sweep_comments)

    with open(filename,'wb') as f:
        pickle.dump(data, f)
else:
    print('Overwrite aborted')

In [9]:
%watermark --iversions

%watermark -p scipy
%watermark --watermark

pandas    : 0.25.3
PIL       : 7.0.0
matplotlib: 3.1.1
IPython   : 7.11.1
numpy     : 1.19.5

scipy: 1.4.1

Watermark: 2.2.0

